In [ ]:
## !pip install --upgrade jupyter-console &> /dev/null
#!pip install bson
#!pip install hyperopt
#!pip install hyperas
#!sudo pip uninstall bson
#!pip install pymongo

In [ ]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as keras

## For saving model
import h5py

from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from keras.models import Sequential
from keras.utils import np_utils
from keras import initializers
from keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [ ]:
def data():
    X_train = np.load("./data/X_train_s.npy")
    X_test = np.load("./data/X_test_s.npy")
    Y_train = np.load("./data/y_1yr_train_s.npy")
    Y_test = np.load("./data/y_1yr_test_s.npy")
    return X_train, Y_train, X_test, Y_test

def model(X_train, Y_train, X_test, Y_test):
    model = Sequential()
    batch_normal_input = "false"
    batch_normal_hidden1 = "false"
    batch_normal_hidden2 = "false"
    batch_normal_hidden3 = "false"
    batch_normal_hidden4 = "false"
  
    
    hidden_layer1_neurons = {{choice([128, 256, 512])}}
    hidden_layer2_neurons = 0
    hidden_layer3_neurons = 0
    hidden_layer4_neurons = {{choice([4, 8, 12, 16, 32])}}

    l2_1 = {{choice([0.01, 0])}}
    l1_1 = {{choice([0])}}

    l2_2 = "false"
    l1_2 = "false"

    l2_3 = "false"
    l1_3 = "false"

    l2_4 = {{choice([0.01])}}
    l1_4 = {{choice([0])}}

    dropout_0 = {{uniform(0.3, 0.7)}}
    dropout_1 = {{uniform(0.3, 0.7)}}
    dropout_2 = "false"
    dropout_3 = "false"
    dropout_4 = {{uniform(0, 0.5)}}

    epochs = {{choice([4, 8, 16, 24, 32, 40])}}
    batch_size = {{choice([4, 8, 12, 16, 32, 64])}}
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=200, min_delta=0, verbose=0, mode='auto')]

    model.add(Dense(256, input_shape=(250,)))
    model.add(Activation('elu'))
    model.add(Dropout(dropout_0))

    model.add(Dense(hidden_layer1_neurons, kernel_regularizer=regularizers.l2(l2_1), activity_regularizer=regularizers.l1(l1_1)))
    model.add(Activation('elu'))
    model.add(Dropout(dropout_1))

    if hidden_layer1_neurons == 512 or hidden_layer1_neurons == 256 or hidden_layer1_neurons == 128:
        if {{choice(['three', 'four'])}} == 'four':
            l2_2 = {{choice([0.01, 0])}}
            l1_2 = {{choice([0])}}
            hidden_layer2_neurons = {{choice([32, 64, 128, 265])}}
            dropout_2 = {{uniform(0, 0.5)}}
            
            model.add(Dense(hidden_layer2_neurons, kernel_regularizer=regularizers.l2(l2_2), activity_regularizer=regularizers.l1(l1_2)))
            model.add(Activation('elu'))
            model.add(Dropout(dropout_2))

    model.add(Dense(hidden_layer4_neurons, kernel_regularizer=regularizers.l2(l2_4), activity_regularizer=regularizers.l1(l1_4)))
    model.add(Activation('elu'))
    model.add(Dropout(dropout_4))

    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))

    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
        optimizer = "adam"
    else:
        optim = rmsprop
        optimizer = "rmsprop"
    
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optim)
    
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs = epochs,
              verbose=0,
              callbacks=callbacks,
              validation_split=0.2)
    
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    
    if acc >= 0.72:
        print('Test accuracy:', acc)
    else:
        print('another one bites the dust')
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
if __name__ == '__main__':
  from hyperas import optim
  from hyperas.distributions import choice, uniform
  from hyperopt import Trials, STATUS_OK, tpe

  X_train, Y_train, X_test, Y_test = data()
    
  best_run, best_model = optim.minimize(model=model,
                                        data=data,
                                        algo=tpe.suggest,                                       
                                        max_evals=300,
                                        trials=Trials(),
                                        notebook_name='modelfinding_1yr_mort')


print(best_run)
print('\n# Evaluate on test data')
results = best_model.evaluate(X_test, Y_test)
print('test loss, test acc:', results)

best_acc = str(results[1])[0:5]

from keras.models import load_model
best_model.save('1yr_Acc_'+best_acc+'_28May2020_1yr_best_model.h5')